In [5]:
from bs4 import BeautifulSoup
from docx import Document

def convert_html_to_docx(html_file, output_docx):
    # Load HTML content from file
    with open(html_file, 'r', encoding='utf-8') as f:
        html_content = f.read()

    # Parse HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Create a new Word document
    doc = Document()

    # Find all text within <div> tags in the HTML
    text_divs = soup.find_all('div')

    # Loop through each div and add its text to the Word document
    for div in text_divs:
        # Add text content of the div
        doc.add_paragraph(div.get_text())

    # Save the Word document
    doc.save(output_docx)

# Example usage:
convert_html_to_docx('index.html', 'output.docx')
